In [10]:
import torch
import torchvision

import os
import torchvision.models as models
from torch.utils.mobile_optimizer import optimize_for_mobile



In [7]:

torch.__version__

'1.10.0+cu113'

In [15]:
model_name='best_model_3_134258.pth'
checkpoint=torch.load(model_name)
model = models.resnet34(num_classes=4)
model.load_state_dict(checkpoint, strict=False)

example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.to('cuda:0')
model.to('cuda:0')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
optimized_model=optimize_for_mobile(traced_script_module)
optimized_model.to('cuda:0')
torch.jit.save(optimized_model, 'optimized_model.pt')

## ptl로 변환하기

In [19]:
optimized_model._save_for_lite_interpreter('new_lite_model.ptl')